In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import *
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [34]:
type = "incone"
data = pd.read_csv(f"./data/{type}_results.csv")


avg = data.groupby(['array_type', 'element_size']).mean().reset_index()
SQ_data = avg[avg["array_type"] == "SQ"]
sliced_data = avg[avg["array_type"] == "sliced"]
mono_data = avg[avg["array_type"] == "mono"]

In [19]:
SQ_data

,array_type,element_size,time
0,SQ,16,0.434240
1,SQ,32,0.468366
2,SQ,48,0.518206
3,SQ,64,0.357967
4,SQ,80,0.370725
...,...,...,...
123,SQ,1984,4.832135
124,SQ,2000,4.289124
125,SQ,2016,4.842349
126,SQ,2032,4.366260


In [35]:
def visualise_position(r1, r2, r3):
    p = figure(title = 'run time of different array types', plot_width = 900, x_axis_label = 'size of each element', y_axis_label = 'avergae time')
    p.line(r1.element_size, r1.time, color = 'blue', legend_label = "SQarray")
    p.line(r2.element_size, r2.time, color = 'red', legend_label = "sliced")
    p.line(r3.element_size, r3.time, color = 'green', legend_label = "monolithic")
    p.legend.location = "top_left"
    show(p)
visualise_position(SQ_data, sliced_data, mono_data)